In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.code import Code
from aiida.orm.data.base import Int, Float, Str
from aiida.orm.querybuilder import QueryBuilder
from aiida.work.run import run, async, submit
from aiida.orm.data.structure import StructureData

import nglview
import numpy as np
import ipywidgets as ipw
from collections import OrderedDict
from IPython.display import display, clear_output, HTML
import nglview

from structure_browser import StructureBrowser
from apps.nanoribbon.workchain import NanoribbonWorkChain

In [ ]:
def on_struct_change(c):
    refresh_structure_view()
    
struct_browser = StructureBrowser()
struct_browser.results.observe(on_struct_change, names='value')    
viewer = nglview.NGLWidget()

clear_output()
display(ipw.VBox([struct_browser, viewer]))

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        #viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    
    node = struct_browser.results.value
    if not node:
        return
    atoms = node.get_ase()
    viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
    viewer.add_unitcell()
    viewer.center_view()

    spin_ups = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==1])
    spin_downs = ",".join([str(i) for i, t in enumerate(atoms.get_tags()) if t==2])

    viewer.add_representation('ball+stick', selection="@"+spin_ups, color='red', aspectRatio=3.0, opacity=0.4)
    viewer.add_representation('ball+stick', selection="@"+spin_downs, color='green', aspectRatio=3.0, opacity=0.4)

In [ ]:
qb = QueryBuilder()
qb.append(Code)
all_codes = list(qb.iterall())

# query AiiDA database for Codes
qb = QueryBuilder()
qb.append(Code)
   
all_nodes = [ node_tuple[0] for node_tuple in qb.iterall() ]
all_nodes.sort(key=lambda n: n.pk)

all_codes = OrderedDict()

for node in all_nodes:
    label = "%s PK: %d"%(node.label, node.pk)
    all_codes[label] = node

default_pw_code = [v for k, v in all_codes.items() if k.lower().startswith("pw")][-1]
default_pp_code = [v for k, v in all_codes.items() if k.lower().startswith("pp")][-1]
default_projwfc_code = [v for k, v in all_codes.items() if k.lower().startswith("projwfc")][-1]
drop_pw_code = ipw.Dropdown(description="pw", options=all_codes, value=default_pw_code)
drop_pp_code = ipw.Dropdown(description="pp", options=all_codes, value=default_pp_code)
drop_projwfc_code = ipw.Dropdown(description="projwfc", options=all_codes, value=default_projwfc_code)
slider_precision = ipw.FloatSlider(description='Precision:', value=1.0, min=0.1, max=3.0, step=0.1)
display(ipw.VBox([drop_pw_code, drop_pp_code, drop_projwfc_code, slider_precision]))

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        pw_code = drop_pw_code.value
        pp_code = drop_pp_code.value
        projwfc_code = drop_projwfc_code.value
        struct = struct_browser.results.value
        precision = Float(slider_precision.value)

        if struct == None:
            print("Please select a structure")
            return

        outputs = submit(NanoribbonWorkChain,
                         pw_code=pw_code,
                         pp_code=pp_code,
                         projwfc_code=projwfc_code,
                         structure=struct,
                         precision=precision)
        print(outputs)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)